In [4]:
### IMPORT STATEMENTS ###

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import string
from models import *
from configs import cfg
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [5]:
### HELPER FUNCTIONS ###

# returns a pandas dataframe of the given file
# fname: string; path to the file
# num_rows: int; number of rows to read (for partial datasets)
# return: pandas Dataframe
def load_data(fname, num_rows = None):
    return pd.read_csv(fname, nrows = num_rows)


# creates a one-hot encoding matrix for the given data
# data: 1d numpy array; list of items/features to encode
# dictionary: dict; mapping from the item to its index
# return: 2d numpy array
def encode_one_hot(data, dictionary):
    # creates the 2d array of zeros
    one_hot_encoding = np.zeros((data.shape[0], len(dictionary)))
    
    # gathers the respective indices of each item
    data_indices = [dictionary[item] for item in data]

    # encodes the 1 for all the items
    one_hot_encoding[range(data.shape[0]), data_indices] = 1
    
    return one_hot_encoding


# converts a one-hot encoding of the reviews into strings
# data: 3d torch list; list of one-hot encoding
# dictionary: dict; mapping from the index to the character
# return: review strings (1d list)
def decode_one_hot_reviews(data):
    extended_char = cfg['valid_char'] + 'SEP'
    decoded = [''.join([extended_char[torch.argmax(c)] for c in review]) for review in data]
    #decoded = [''.join([cfg['valid_char'][torch.argmax(c)] 
                #if torch.argmax(c) < cfg['valid_char_len'] 
                #else '' for c in review]) for review in data]
    
    return decoded


# cleans and processes (feature encoding) the training data
# orig_data: pandas Dataframe; raw data that is outputted from load_data
# returns: features (2d numpy array; one-hot), labels (1d numpy array of strings), beer dictionary (dict)
def process_train_data(orig_data):
    print ("Processing training data")
    print ("Original data shape: " + str(orig_data.shape))
    
    # takes the relevant columns
    data = orig_data[['beer/style', 'review/overall', 'review/text']].copy()
    
    # --- DATA CLEANING ---
    
    # drops the rows with missing data
    data.replace('', np.nan, inplace = True)
    data.dropna(inplace = True)
    data.reset_index(drop = True, inplace = True)

    # sets all characters to lower case
    data['beer/style'] = data['beer/style'].str.lower()
    data['review/text'] = data['review/text'].str.lower()

    # converts all whitespace (space, tabs, newlines, etc.) into spaces
    whitespace_regex = '[' + string.whitespace + ']'
    data['review/text'] = data['review/text'].str.replace(whitespace_regex, ' ', regex = True)

    # removes all invalid characters
    invalid_char_regex = '[^' + cfg['valid_char'] + ']'
    data['review/text'] = data['review/text'].str.replace(invalid_char_regex, '', regex = True)
    
    print ("Data shape after cleaning: " + str(data.shape))
    
    # --- DATA PROCESSING ---
    
    # creates a list of beer and a dictionary to map a beer style to an index
    beer_list = data['beer/style'].unique()
    beer_to_index = dict(zip(beer_list, range(beer_list.shape[0])))
    print ("Number of unique beers: " + str(beer_list.shape[0]))

    # creates the input features
    beer_encoding = encode_one_hot(data['beer/style'].values, beer_to_index)
    score_encoding = data['review/overall'].values
    score_encoding = score_encoding.reshape(score_encoding.shape[0], 1)
    input_features = np.hstack((beer_encoding, score_encoding))
    print ("Input feature matrix shape: " + str(input_features.shape))
    
    # creates the labels
    labels = data['review/text'].values
    print ("Labels matrix shape: " + str(labels.shape))
    
    return input_features, labels, beer_to_index


# updates the configurations based on the results of the processed dataset
def update_configurations(feature_length):
    # sets the models' input dimensions to the size of features (beer style + score) + character encoding
    cfg['input_dim'] = feature_length + cfg['output_dim']

    
# splits the dataset + labels into a training and validation set
# features: numpy array
# labels: numpy array
# percent_training: float; percentage (from 0.0 to 1.0) of data to be used for training
# returns: training features, training labels, validation features, validation labels (all numpy arrays)
def train_valid_split(features, labels, percent_training):
    # gets the index of where to split
    training_last_index = int(percent_training * features.shape[0])

    x_train = features[:training_last_index]
    y_train = labels[:training_last_index]

    x_valid = features[training_last_index:]
    y_valid = labels[training_last_index:]
    
    print ("Training set size: " + str(x_train.shape[0]))
    print ("Validation set size: " + str(x_valid.shape[0]))
    
    return x_train, y_train, x_valid, y_valid
    

# cleans and processes (feature encoding) the testing data
# orig_data: pandas Dataframe; raw data that is outputted from load_data
# dictionary: dict; mapping from the beer style to its index (output of process_train_data)
# returns: features (2d numpy array; one-hot)
def process_test_data(orig_data, dictionary):  
    print ("Processing the testing data")
    print ("Original data shape: " + str(orig_data.shape))
    
    # takes the relevant columns
    data = orig_data[['beer/style', 'review/overall']].copy()
    
    # --- DATA CLEANING ---
    
    # sets all characters to lower case
    data['beer/style'] = data['beer/style'].str.lower()
    
    # --- DATA PROCESSING ---
    
    # creates the input features
    beer_encoding = encode_one_hot(data['beer/style'].values, dictionary)
    score_encoding = data['review/overall'].values
    score_encoding = score_encoding.reshape(score_encoding.shape[0], 1)
    input_features = np.hstack((beer_encoding, score_encoding))
    print ("Input feature matrix shape: " + str(input_features.shape))
        
    return input_features


# pads the reviews so that all reviews in the set have an equal size
# and adds the <SOS> and <EOS> tags to the beginning and end of the reviews
# orig_data: 2d list of ints; list of reviews with the characters converted to their respective indices
# outputs: 2d numpy array of ints; padded reviews with the characters as indices
def pad_data(orig_data):
    # defines the character indices for the <SOS>, <EOS>, and <PAD> tags
    sos_tag_index = cfg['valid_char_len']
    eos_tag_index = sos_tag_index + 1
    pad_tag_index = eos_tag_index + 1
    
    # finds the longest review length
    review_lengths = [len(review) for review in orig_data]
    longest_review_length = np.max(review_lengths)
    
    # pads the reviews and adds the <SOS> and <EOS> tags
    padded_reviews = []
    for review in orig_data:
        pad_length = longest_review_length - len(review)
        padded_review = [sos_tag_index] + review + [eos_tag_index] + [pad_tag_index] * pad_length
        padded_reviews.append(padded_review)
        
    return np.array(padded_reviews)


def train(model, model_name, criterion, optimizer, computing_device, x_train, y_train, x_valid, y_valid, cfg):
    train_loss = []
    valid_loss = []
    valid_bleu = []

    start_time = time.time()
    
    softmax = nn.LogSoftmax(dim = 1)
    bleu_smoothing = SmoothingFunction()
    
    early_stop_count = 0
    min_loss = 100

    for epoch in range(1, cfg['epochs'] + 1):

        print ('----- Epoch #' + str(epoch) + ' -----')

        start_index = 0
        end_index = cfg['batch_size']

        losses = []

        print ('----- Training -----')
        while start_index < len(x_train):
            # takes the minibatch subset
            batch_x = x_train[start_index:end_index]
            batch_y = y_train[start_index:end_index]

            # converts the reviews char -> index
            indexed_reviews = [[char_to_index[c] for c in review] for review in batch_y]

            # pads the reviews
            padded_reviews = pad_data(indexed_reviews)

            # converts the review to a one-hot encoding
            # and concatenates this to the input features
            one_hot_length = cfg['output_dim']
            final_batch_x = []
            for features, reviews in zip(batch_x, padded_reviews):
                for char_index in reviews[:-1]:
                    one_hot_encoding = np.zeros(one_hot_length)
                    one_hot_encoding[char_index] = 1
                    final_features = np.hstack((features, one_hot_encoding))
                    final_batch_x.append(final_features)

            # converts the final array into a numpy array
            final_batch_x = np.array(final_batch_x)

            # resizes the flattened array into batch_size x sequence_length x feature_length
            final_batch_x.resize(padded_reviews.shape[0], padded_reviews.shape[1] - 1, final_batch_x.shape[1])

            # converts final input array to tensor
            final_batch_x = torch.from_numpy(final_batch_x).float().to(computing_device)

            # zeros the gradients
            optimizer.zero_grad()

            # passes the final input array to the model's forward pass
            outputs, _ = model(final_batch_x)
            soft_outputs = softmax(outputs)

            # prints the actual reviews vs the predicted reviews
            actual_reviews = batch_y
            predicted_reviews = decode_one_hot_reviews(soft_outputs)
            
            for i in range(1):
                print ("Actual Review: " + actual_reviews[i])
                print ("Predicted Review: " + predicted_reviews[i])
            
            # reshapes the outputs to N x feature_length (for the loss function)
            outputs = outputs.contiguous().view(-1, outputs.shape[2])

            # creates the targets and reshapes it to a single dimension
            targets = torch.from_numpy(padded_reviews[:, 1:]).long().to(computing_device)
            targets = targets.contiguous().view(-1)

            # passes the outputs and targets to the loss function and backpropagates
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            print("E" + str(epoch) + "T Batch start index: " + str(start_index) + " | Loss: " + str(loss.item()))
            print("Time elapsed: " + str(time.time() - start_time))

            start_index = end_index
            end_index += cfg['batch_size']
            
            # stops training when the remaining data count is less than a minibatch
            if end_index > len(x_train):
                break


        train_loss.append(np.mean(losses))
        torch.save(model, model_name + "_e" + str(epoch) + ".pt")
        print()

        print ('----- Validating -----')
        start_index = 0
        end_index = cfg['batch_size']

        losses = []
        bleus = []

        with torch.no_grad():

            while start_index < len(x_valid):
                # takes the minibatch subset
                batch_x = x_valid[start_index:end_index]
                batch_y = y_valid[start_index:end_index]

                # converts the reviews char -> index
                indexed_reviews = [[char_to_index[c] for c in review] for review in batch_y]

                # pads the reviews
                padded_reviews = pad_data(indexed_reviews)

                # converts the review to a one-hot encoding
                # and concatenates this to the input features
                one_hot_length = cfg['output_dim']
                final_batch_x = []
                for features, reviews in zip(batch_x, padded_reviews):
                    for char_index in reviews[:-1]:
                        one_hot_encoding = np.zeros(one_hot_length)
                        one_hot_encoding[char_index] = 1
                        final_features = np.hstack((features, one_hot_encoding))
                        final_batch_x.append(final_features)

                # converts the final array into a numpy array
                final_batch_x = np.array(final_batch_x)

                # resizes the flattened array into batch_size x sequence_length x feature_length
                final_batch_x.resize(padded_reviews.shape[0], padded_reviews.shape[1] - 1, final_batch_x.shape[1])

                # converts final input array to tensors
                final_batch_x = torch.from_numpy(final_batch_x).float().to(computing_device)

                # passes the final input array to the model's forward pass
                outputs, _ = model(final_batch_x)
                soft_outputs = softmax(outputs)
                
                # prints the actual reviews vs the predicted reviews
                actual_reviews = batch_y
                predicted_reviews = decode_one_hot_reviews(soft_outputs)
                
                for a, p in zip(actual_reviews, predicted_reviews):
                    bleus.append(sentence_bleu(a.split(), p.split(), weights = [1.0], smoothing_function = bleu_smoothing.method1))
                    
                for i in range(1):
                    print ("Actual Review: " + actual_reviews[i])
                    print ("Predicted Review: " + predicted_reviews[i])


                # resizes the outputs to N x feature_length (for the loss function)
                outputs = outputs.contiguous().view(-1, outputs.shape[2])

                # creates the targets and reshapes it to a single dimension
                targets = torch.from_numpy(padded_reviews[:, 1:]).long().to(computing_device)
                targets = targets.contiguous().view(-1)

                # passes the outputs and targets to the loss function
                loss = criterion(outputs, targets)

                losses.append(loss.item())

                print("E" + str(epoch) + "V Batch start index: " + str(start_index))
                print("Loss: " + str(loss.item()) + " | BLEU score: " + str(np.mean(bleus)))
                print("Time elapsed: " + str(time.time() - start_time))

                start_index = end_index
                end_index += cfg['batch_size']
                
                # 
                if end_index > len(x_valid):
                    break


        average_loss = np.mean(losses)
        valid_loss.append(average_loss)
        valid_bleu.append(np.mean(bleus))
        
        print()

        # checks for early stopping when the validation loss is higher for x consecutive epochs
        if average_loss >= min_loss:
            early_stop_count += 1

            if early_stop_count >= cfg['early_stop']:
                break

        else:
            early_stop_count = 0
            min_loss = average_loss
            
    return train_loss, valid_loss, valid_bleu
    
    
def process_results(model_name, train_loss, valid_loss, valid_bleu):
    # summarizes the results
    print (model_name + " Results:")
    print ("Training Loss: " + str(train_loss))
    print ("Validation Loss: " + str(valid_loss))
    print ("Validation Bleu Score: " + str(valid_bleu))

    # graphs the loss curves
    plt.clf()
    plt.plot(range(len(train_loss)), train_loss, 'b--', label = 'Training Loss')
    plt.plot(range(len(valid_loss)), valid_loss, 'r--', label = 'Validation Loss')

    plt.grid(True)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(model_name + " Loss Curve")
    plt.legend(loc="upper right")

    plt.savefig(model_name + " Loss.png")
    
    
    # graphs the bleu score curve
    plt.clf()
    plt.plot(range(len(valid_bleu)), valid_bleu, 'r--', label = 'Validation Bleu Score')
    
    plt.grid(True)
    plt.xlabel("Epoch")
    plt.ylabel("Bleu Score")
    plt.title(model_name + " Bleu Score Curve")
    plt.legend(loc="lower right")

    plt.savefig(model_name + " Bleu Score.png")
    
    
def sample(outputs, temperature):
    #logged = np.log(outputs) / temperature
    #exped = np.exp(logged)
    #sigmoided = exped / np.sum(exped)
    distribution = outputs.div(temperature).exp()
    print ('distribution')
    print (distribution)
    return torch.multinomial(distribution, 1)[0]
    
    
    #return np.random.multinomial(1, sigmoided)
    
def generate(model, x_test, cfg):
    # TODO: Given n rows in test data, generate a list of n strings, where each string is the review
    # corresponding to each input row in test data.
    
    predicted_reviews = []
    extended_char = cfg['valid_char'] + 'SEP'
    
    start_index = 0
    end_index = cfg['batch_size']
    
    start_time = time.time()
    softmax = nn.Softmax(dim = 2)

    print ('----- Testing -----')
    with torch.no_grad():
        while start_index < len(x_test):
            # takes the minibatch subset
            batch_x = x_test[start_index:end_index]

            # sets the outputs as the <SOS> tag for each review
            outputs = np.zeros((cfg['batch_size'], cfg['output_dim']))
            outputs[:, cfg['valid_char_len']] = 1

            # initializes the states
            ht = None
            ct = None

            # initializes the predicted sentences
            sentences = [[] for _ in range(cfg['batch_size'])]

            # samples the next character until all are either <EOS> or <PAD> (all 1s are in the last 2 columns)
            while np.sum(outputs[:, -2:]) < cfg['batch_size'] and len(sentences[0]) < cfg['max_len']:
                # concatenates the outputs (previous characters) to the metadata to get the inputs
                final_batch_x = np.hstack((batch_x, outputs))

                # resizes the array into batch_size x sequence_length (1) x feature_length
                final_batch_x.resize(final_batch_x.shape[0], 1, final_batch_x.shape[1])

                # converts final input array to tensor
                final_batch_x = torch.from_numpy(final_batch_x).float().to(computing_device)

                # passes the final input array to the model's forward pass
                if isinstance(model, bLSTM):
                    outputs, (ht, ct) = model(final_batch_x, ht, ct)

                else:
                    outputs, ht = model(final_batch_x, ht)
                    

                #outputs = np.array([sample(c[0], cfg['gen_temp']) for c in outputs])
                #outputs = outputs.numpy()
                #outputs = outputs.reshape(outputs.shape[0], outputs.shape[2])
                #outputs = outputs / cfg['gen_temp']
                #outputs = np.exp(outputs)
                #outputs = outputs / np.sum(outputs, axis = 1)[:, None]
                
                outputs = outputs.div(cfg['gen_temp'])
                
                outputs = softmax(outputs)
                
                outputs = outputs.contiguous().view(outputs.shape[0], outputs.shape[2])
                
                outputs = torch.multinomial(outputs, 1)
                outputs = outputs.numpy()
                sentences = np.hstack((sentences, outputs))
                
                
                indexes = outputs.reshape(cfg['batch_size'])
                outputs = np.zeros((cfg['batch_size'], cfg['output_dim']))
                outputs[range(cfg['batch_size']), indexes] = 1


            decoded = [''.join([extended_char[int(c)] for c in review]) for review in sentences]
            predicted_reviews.append(decoded)
            '''
            for s in sentences:
                decoded = ''
                for c in s:
                    print(type(c))
                    decoded = decoded + extended_char[c]

                predicted_reviews.append(decoded)
            '''

            #print ("Predicted Review: " + predicted_reviews[start_index])
            print ("Predicted Review: " + decoded[0])

            print("Batch start index: " + str(start_index))
            print("Time elapsed: " + str(time.time() - start_time))

            start_index = end_index
            end_index += cfg['batch_size']

            if start_index == len(x_test):
                break

            # case when the remaining data count is less than a minibatch
            if end_index > len(x_test):
                # adjusts the start and end indices to make the last subset the size of a minibatch
                end_index = len(x_test)
                start_index = end_index - cfg['batch_size']

                # removes the last few predictions to avoid duplicates
                predicted_reviews = predicted_reviews[:start_index]

    print()
    return predicted_reviews
    
    
def save_to_file(outputs, fname):
    # TODO: Given the list of generated review outputs and output file name, save all these reviews to
    # the file in .txt format.
    raise NotImplementedError


In [6]:
### MAIN FUNCTION ###

train_data_fname = "Beeradvocate_Train.csv"
test_data_fname = "Beeradvocate_Test.csv"
out_fname = "Output_Reviews.txt"

# loads the data
train_data = load_data(train_data_fname, cfg['num_data'])
#test_data = load_data(test_data_fname, 50)

# processes the data to get the train, valid, and test sets
train_data, train_labels, beer_to_index = process_train_data(train_data)
x_train, y_train, x_valid, y_valid = train_valid_split(train_data, train_labels, cfg['train_percentage'])
#x_test = process_test_data(test_data, beer_to_index)

# updates the configurations based on the processed data
update_configurations(x_train.shape[1])

# creates the dictionaries to map a character to its index in a one-hot encoding
char_to_index = dict(zip(list(cfg['valid_char']), range(cfg['valid_char_len'])))

# gets the computing device (either cuda or cpu)
if torch.cuda.is_available():
    computing_device = torch.device("cuda")
    cfg['cuda'] = True
    
else:
    computing_device = torch.device("cpu")
    cfg['cuda'] = False

Processing training data
Original data shape: (5000, 14)
Data shape after cleaning: (4999, 3)
Number of unique beers: 78
Input feature matrix shape: (4999, 79)
Labels matrix shape: (4999,)
Training set size: 3999
Validation set size: 1000


In [7]:
# defines the hyperparameters
model_number = '1'

cfg['hidden_dim'] = 10
cfg['layers'] = 1

In [ ]:
# trains the LSTM model
model = bLSTM(cfg).to(computing_device)
optimizer = optim.Adam(model.parameters(), cfg['learning_rate'])
criterion = nn.CrossEntropyLoss()

train_loss, valid_loss, valid_bleu = train(model, "LSTM" + model_number, criterion, optimizer, computing_device, 
                                           x_train, y_train, x_valid, y_valid, cfg)

process_results("LSTM Model " + model_number, train_loss, valid_loss, valid_bleu)

predicted_reviews = generate(model, x_train, cfg)
#print (predicted_reviews)

----- Epoch #1 -----
----- Training -----
Actual Review: a lot of foam. but a lot. in the smell some banana, and then lactic and tart. not a good start. quite dark orange in color, with a lively carbonation (now visible, under the foam). again tending to lactic sourness. same for the taste. with some yeast and banana.
Predicted Review: sbzzz999l999llS-.d99999z9S-99999l99e899999el9mll888z999l99zl99999991998l9998qS-999999SSSll99S8qS-l.1999998899898889999999z999.99999999ll9l9998mll89999999z99l991.99l9dlll9999l999lll--SSS999999l99999S99999919?Sd9989?S-d4?89999999l999999S-.999999e89lll99999l9mll888-999!nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

Actual Review: i bought a six-pack of caldera pale ale at a grocery store. poured in to a pint glass.  pours a rich gold color and is ever so slightly hazy. creamy white head caps the beer and is quite resilient. the head dissipates in ridges and valleys, finally getting down to small islands of foam on the surface of the brew..  smell is fairly faint. some hoppy fruity esters perhaps, but generally unremarkable.  taste is a bitter flavor, like sour citrus. a little malt flavor sneaks in too. strong flavors for a pale, almost approaching what one would expect from an ipa.  mouthfeel is light, crisp and clean, but dry. not watery, but light on the tongue.   a drinkable pale ale, but i think the sour citrus flavor is a bit over done for the style. i'd have this again though. the packaging (cans) are great for camping, tailgates and the like, but it's a bit overpriced in my area at $8 a sixer.
Predicted Review: :s(:z:S$9(--q?..ll88-z9(8S-zz8--z9zz$-zz$-q------S88m$qqSq8S-zz:e80$98$qS$-$z9

E1T Batch start index: 192 | Loss: 3.9042553901672363
Time elapsed: 17.283617973327637
Actual Review: a- black with thin tan head. good lace  s- sweet, slight hint of estersfruit, roasted malt...typical porter nose  t- similar to nose. initial slight esteryfruitiness transitions smoothly to roasty-sweet, coffeechocolate finish. one of the best balances of hops-malt i've had in a porter.  m- full bodied, silky and slightly creamy. perfect for the style.  d- very sessionable....very close to perfect for the style
Predicted Review: zz:$zz9989.9qz(qz999qSm9zz99S-SS2l9z99899q.(q.9mql(q99Szq9z99q9z99z?q9?q9qd9qd(q9Sqq999eS9qSSSq..9?S99zz9q9?99zq999q.$q9??99q9q99zzq9S-999q9?99q99Szq99?q9?.99d9q9qm?qeqqqbq9q9dm?9q4diqzzz$q99q9Sqq..d.mmql(Si999m9zzSSz9q999999?5S(2zz9z99qzz$zz?q93dz9z88?9.99zzz9.dS9q99..m9z9l999999zz9q9?--9).$9.zz$zzl99ll(d999.99m99q99Szq9z9Sq99e.S-zz9999q99.99qzz9qq.9z--99.$.l9.9q9?q?dmSmzz--SS.l9.999zq99q99zz9999q99.99qzz9qq.9zzllnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

E1T Batch start index: 320 | Loss: 3.8424668312072754
Time elapsed: 26.113123893737793
Actual Review: the can oozed with creamy foam before i poured it. needledd to say the head was soft and creamy. it sat atop the cloudy caramelorange body for a good amount of time before sinking. it left an excellent wall of lace, topping it all off.  the nose had a good complex hop aroma, just a tad short on intensity. hints of pineapple, fresh tomato vines, fresh grass, and a dash of astringency and metallic twang make up the one-sided hop aroma.  the flavor is excellent. it takes what the nose started and cranks it up a notch. passion fruit sweetness meets a bold herbal grassy profile, and the two work together great. malt knows its roll.... the shy pale toasty backbone. the finish is bone dry with grassy bitterness.   this one is pretty dry right from the get-go. the carbonation is perfect and keeps it feeling on the light and crisp side. my mouth needs the constant reinforcement of another drink

E1T Batch start index: 448 | Loss: 3.8052375316619873
Time elapsed: 34.654152631759644
Actual Review: pour was a deep copper with a slight hazy ruby tint. fluffy cloud like head with decent retention and lacing  huge malts with a slight citrus and floral hop scent. mainly orange and pineapple.  bitter up front, a strong pine bitterness with bits of earthy and citrus bitterness. a bit of white grapefruit, orange, and sweet biscuit malts.   very full bodied with proper carbonation. the perfect bite to accompany the flavors.  not bad, has a lot of deep bitterness and is a solid ipa. i would prefer some floral flavors but that is just me. the price here would have me thinking twice about getting more but i would have no issues drinking this again.
Predicted Review: :::de(.9q(d(llll99llll99.9ql(9(d99Slq($9zz(9d..(q99qS)99dll.(S9ldl(99999zl9l9.9ql((l98mq999q9mq9d9(99l(99999S)($dSl(dS9qq(.9ql(9(d99Slq(S9q?dd(d9l(l9l999(zll$dS8mqS)dS999z(l949Sl99ll(l9999ll9l--(.9qq9?9dl(l9llql(d(dqqd9S(l9999.9

...

Actual Review: notes from 110211  *bomber w a 10oz nonic   a: pours a shiny black licorice color with a sort of lighter crimson edge to it. almost no head but just a rim of turbinado sugar color foam on topmaybe half a finger at most with low retention and some dotted lace. just dark and oily looking.   s: smells a bit woody with an earth and spice mix going on. vanilla beans, caramelized malts, and a hint of chocolate. some alcohol aromas exude. really rather complex array of aromas here.   t: wow this is a big robust stout with lots going on. tastes of charred roasty grains, earthy, with a distinct whisky note. a touch harsh but the whisky barrel treatment is rather subtle and not completely overbearing here. dark chocolates. slightly sweeter with some hints of vanilla. hops start midway and spike the back of the palate bringing a sort of dark fruit component to the brew. a little bit of alcohol heat and a dry roast coffee aftertaste.  m: has a rather smooth mouthfeel overall. medium

Actual Review: sampled at the cleveland beer fest 51411. poured black with a tan colored head that has enough carbonation to leave a small ring that lasts. notes of, whiskey, and heavily roasted malt and burnt chocolate fill the nose. this is a full bodied stout with yeast flavors enhanced by malt, burnt chocolate, coffee and a lingering hint of hop on finish. a little hot on the alcohol. i would love to let a bottle sit for a year or so.
Predicted Review: 3((rr(r(((((irr(l(lrr((((((((((((ii(rl(llirr(r(r(((((lr((lir((((rr(ll((((((((((((rri(rr(((((r(r(lr((((rilrr(i(((((rr((((ir((((lrr(irri((((iii(rriri(((((lrrirrrr((((((((rr(r(((((irr(lr(((((((((((i(lr(l((((r((l((((rr((((ri(irri(li(r((l((((((lrr((irri(llir(rrr(i(l((r((i(((rr(ll(((((lr((l((r(ri(lr(l((((rr(lll(lr((((((((lrrr(lrr(rrri((((r(((((((lrriri((((lii(r(r((((((irr(((l(r((((lll(r(((((rr((((((((((((ii(r((lirl((((((rr(((((i(irrrrrrrnnxPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP

Actual Review: 22 oz bomber at 50 degrees into a stone irs tulip.  a: it pours pure opaque black with a two-finger dome of chocolate milk colored creaminess. it holds up very well and the lacing is exceptional.  s: milk chocolate, sweet dark fruits, some anise-licorice notes, and light to average roastiness.  t: strong milk chocolate, sweet figs and dates, moderate and semi-citrusy hoppiness, and anise and licorice late. there is a lack of roastiness in this that makes it refreshingly unique.  m: this is the most velvety-smooth imperial stout i've ever had. b.o.r.i.s. on nitro-tap might not be much smoother.  d: it goes down easily and i feel as if i could go for another.
Predicted Review: 330((((((llrr(ri(ri((r((rri(lrirrr(iirrr(lririillrir(rr(lirrrrri(rrrr((rrrrr((((lr(llirrr(iirrllrrrr(((lr((((lrrll(rir(lllr(ll(((r((lrrrllrriiiriirrr((((lr(rr(r(lr(((((((irr(((llrr(lirriirriirrr(i((iiriirr(lrrll(rirr(iirrirrr(r(((lliiiririrrrrlirr(lirrlirrrririirrrr((lrri(ir(rrrrr(((((riiirriiir(iiri

E3T Batch start index: 32 | Loss: 2.0162086486816406
Time elapsed: 788.640909910202
Actual Review: i received this one in a bif recently and it was the first thing i grabbed out of the fridge when i got home on friday. served cold and poured into a pint glass, this one was consumed on 07312009.  appearance - one of the best pours from a pale ale you could possibly ask for. seriously. big head of white foam on top that barely falls back down to earth. wonderful clear amber color that really just looks the part perfectly. lots of carbonation creeping up the glass throughout and a really nice coating of spider like webbing.  smell - aroma is lighter then i would have thought. nice light grains, a touch of piney hops. as it warms a bit of citrus starts to come though, but again, lighter then i would have thought.  taste - crisp, clean, the first sip has a biting touch to it as the pine and citrus start to roll through. nice grains come in mid palate and then it rolls right back into a nice

E3T Batch start index: 128 | Loss: 1.542265772819519
Time elapsed: 797.0273282527924
Actual Review: 12oz can. a) pours with a finger thick, white head that takes awhile to settle. slightly hazy body. active carbonation. gold in color. nice sticky lacing on the drink down. s) pink grapefruit. a hint of evergreen. t) the initial flavors are washed out by the fizz. once beyond that some dry nutty-ness liken to walnut. grapefruit rind. a mellow sweetness throughout but not the main stay. a nice hop bite in the finish, but seems a bit too much for a pale ale. m) quite fizzy. medium body. dry. d) a decent brew. i'd have another.
Predicted Review: 3300(((((((((((((((((lir((((l((((((rrlr(((rr(((r(((((r(((((((((((rr(((((((((i((((((l(r((((r(((((((((((((lrr(((((((((l((((((((((((((((((((lll((ilirr((((l(((((((r((((l(((((((((((((r(((((((((((rii(((rr(((((((r(((((((((((((r((l(rirr(((((r(((((((((((r(((((((((((r(((l(((((((((((r(((((r(((((l((((r((((ir(((((((lr((((((((((((i(((((((((ri((lr(((((((((((((((((

In [ ]:
# trains the GRU model
model = bGRU(cfg).to(computing_device)
optimizer = optim.Adam(model.parameters(), cfg['learning_rate'])
criterion = nn.CrossEntropyLoss()

train_loss, valid_loss, valid_bleu = train(model, "GRU" + model_number, criterion, optimizer, computing_device, 
                                           x_train, y_train, x_valid, y_valid, cfg)

process_results("GRU Model " + model_number, train_loss, valid_loss, valid_bleu)

predicted_reviews = generate(model, x_test, cfg)
print (predicted_reviews)

In [ ]:
'''
train(model, X_train, y_train, X_valid, y_valid, cfg) # Train the model
outputs = generate(model, X_test, cfg) # Generate the outputs for test data
save_to_file(outputs, out_fname) # Save the generated outputs to a file
'''